In [ ]:
import os
import random

import torch
from xml.etree import ElementTree as ET
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from pathlib import Path
import glob
import numpy as np
from PIL import Image
import torchvision
import torchvision.models
from tqdm.notebook import tqdm
from torch import nn
import shutil
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patheffects as pe
import torchvision.transforms as transforms
import yaml
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
from lightning.pytorch.callbacks import LearningRateFinder
from lightning.pytorch import Trainer, LightningModule
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [23]:
# Путь к данным обучения
data_path = "./Emergency Vehicles Russia.v3i.yolov8"
# data_path = "./Emergency Vehicles Russia.v3i.voc"

model_type = "yolo"
# model_type = "ssd"

# Выбор устройства для работы модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Проверка доступности гпу
# print(device)

In [24]:
def seed_everything(seed):
    """
        Фисксация всех сидов в программе для корректного
        сравнения оптимизаторов и обучаемых моделей

        :seed: число для фиксации сидов
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed = 42
seed_everything(seed)

In [25]:
def ReadClassDict(path):
    '''
        Загружаем список классов изображений из yaml файла

    :param path:
    :return:
    '''

    with open(path, "r") as f:
        file = yaml.safe_load(f)
        class_list = file["names"]
        class_dict = dict(enumerate(class_list, start=0))

    return class_dict

# class_dict = ReadClassDict(data_path + "/data.yaml")
class_dict = ReadClassDict("C:\\Users\\Alex\\PycharmProjects\\Cursach3\\Emergency Vehicles Russia.v3i.yolov8\\data.yaml")


# print(class_dict) # {0: 'ambulance', 1: 'fire fighting vehicle', 2: 'intensive care unit', 3: 'non emergency car', 4: 'police car'}

In [26]:
def get_txt_data(image_path, class_dict, model_type, original_image_size_x, original_image_size_y):
    """
        Получение списка данных по всем bbox'ам на изображении

        :image_name: имя файла
        :path: путь (test train valid)
        :class_dict: словарь с расшифровкой классов
        :return:
    """
    # TODO заменить на убирание всех форматов лишних, не только jpg
    if image_path.rfind(".jpg"):
        image_path = image_path.replace(".jpg", "")

    # читаем соответствующий изображению txt или xml
    if model_type == "yolo":
        
        with open(str(image_path).replace("images", "labels") + ".txt", "r") as f:
            # итерация через bbox'ы объектов в файле и их сохранение в один массив
            bboxes = []
            for line in f:
                data = line.split(sep=" ") # class_id center_x center_y width height
                # print(data)
                center_x = float(data[1])
                center_y = float(data[2])
                width = min(float(data[3]), original_image_size_x)
                height = min(float(data[4]), original_image_size_x)
                card_class = int(data[0]) # номер класса
    
                res = (center_x, center_y, width, height, card_class)
                bboxes.append(res)
            return bboxes
            
    elif model_type == "ssd":
        
        with open(str(image_path).replace("images", "labels") + ".xml", "r") as f:
            # итерация через bbox'ы объектов в файле и их сохранение в один массив
            # read xml
            tree = ET.parse(f)
            print(tree)
            treeroot = tree.getroot()
            # iterate over bboxes
            bboxes = []
            class_dict = {'ambulance': 0, 'fire fighting vehicle': 1, 'intensive care unit': 2, 'non emergency car': 3, 'police car': 4}
            for member in treeroot.findall("object"):
                # you need: xmin ymin xmax ymax, class in dict
                x_min = min(int(member.find("bndbox/xmin").text), original_image_size_x)
                y_min =  min(int(member.find("bndbox/ymin").text), original_image_size_y)
                x_max =  min(int(member.find("bndbox/xmax").text), original_image_size_x)
                y_max =  min(int(member.find("bndbox/ymax").text), original_image_size_y)
                card_class = class_dict[member.find("name").text]
        
                res =  (x_min, y_min, x_max, y_max, card_class)
                print(res)
                
                bboxes.append(res)
            return bboxes


# Проверка работы
# print(get_txt_data("Emergency Vehicles Russia.v1i.yolov8/train/images/0c273b07-_44_PNG.rf.24d358e5a9859dde93cf9823287e0f94.jpg", class_dict, "yolo"))
# print(get_txt_data("./Emergency Vehicles Russia.v1i.yolov8/train/images\-10_PNG.rf.0f13a65f5ba37714e528b76181b3dee8.jpg", class_dict, "yolo"))

# print(get_txt_data("Emergency Vehicles Russia.v1i.voc/test/0df5031f-_2_PNG.rf.a37c60000bb15a0b8c4b135ade9c17f0.jpg", class_dict, "ssd"))

In [29]:
def collate_fn(batch, downsample=32, model_type="ssd", original_image_size=640):
    imgs, batch_boxes = map(list, (zip(*[(b["image"], b["bboxes"]) for b in batch])))

    imgs = torch.stack(imgs)
    b, _, h, w = imgs.shape

    target = imgs.new_zeros(b, 6, h // downsample, w // downsample)

    # Add sample index to targets
    for i, boxes in enumerate(batch_boxes):
        # get_txt_data -> res = (center_x, center_y, width, height, card_class)
        
        if model_type == "yolo":
            center_x, center_y, width, height, classes = map(
            torch.squeeze, torch.split(imgs.new_tensor(boxes), 1, dim=-1)
            )
            classes = classes.int()
        
            # Нормализуйте ширину и высоту, поделив на ширину и высоту исходного изображения
            w_box = (width)/original_image_size # ширина бокса отнормированная
            h_box = (height)/original_image_size # высота бокса отнормированная
    
            cx_idx = (center_x // downsample).to(torch.long) # индекс центра на карте признаков размера 20x20. Это будут как раз координаты пикселя, куда мы запишем параметры коробки
            cy_idx = (center_y // downsample).to(torch.long) # .to(torch.long)
    
            cx_box = center_x - (center_x//downsample)*downsample # сдивиги относительно cx_idx
            cy_box = center_y - (center_y//downsample)*downsample # сдивиги относительно cy_idx

            target[i, :, cy_idx, cx_idx] = torch.stack(
                [cx_box, cy_box, w_box, h_box, torch.ones_like(cx_box), classes]
            )
            
        # elif model_type == "ssd":
        #     # get_txt_data -> res =  (x_min, y_min, x_max, y_max, card_class)
        #     x_min, y_min, x_max, y_max, classes = map(
        #     torch.squeeze, torch.split(imgs.new_tensor(boxes), 1, dim=-1)
        #     )
        #     classes = classes.int()

        #     cx_idx = tensor.new_full((3, 4), (original_image_size//downsample)).to(torch.long) # TODO (посчитайте индекс центра на карте признаков размера 20x20. Это будут как раз координаты пикселя, куда мы запишем параметры коробки)
        #     cy_idx = (original_image_size//downsample).to(torch.long)
            
        #     target[i, :, cy_idx, cx_idx] = torch.stack(
        #         [x_min, y_min, x_max, y_max, torch.ones_like(x_min), classes]
        #     )

    return {"image": imgs, "target": target}

In [30]:
test_col = collate_fn([train_ds[20]], model_type="ssd", original_image_size =640)
print(test_col["target"].shape)

(328, 147, 594, 580, 2)
torch.Size([1, 6, 20, 20])


In [31]:
# Класс датасета для доступа к данным во время обучения
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, *, transform, root=data_path, data_type_path="train", model_type, seed=seed):
        self.root = Path(root)
        self.transform = transform
        self.model_type = model_type

        assert self.root.is_dir(), f"No data at `{root}`"

        # Проверка на корректный путь TODO
        if data_type_path not in ["train", "test", "valid"]:
            data_type_path = "train"
        if model_type == "yolo":
            self.filenames = np.array(glob.glob(root + "/" + data_type_path + "/images/*"))
        elif model_type == "ssd":
            self.filenames = np.array(glob.glob(root + "/" + data_type_path + "/*.jpg"))

        # # {0: 'ambulance', 1: 'fire fighting vehicle', 2: 'intensive care unit', 3: 'non emergency car', 4: 'police car'}
        # self.class_dict = ReadClassDict("./" + data_path + "/data.yaml")
        # # {'ambulance': 0, 'fire fighting vehicle': 1, 'intensive care unit': 2, 'non emergency car': 3, 'police car': 4}
        # self.class_dict_inverted = {v: k for k, v in self.class_dict.items()}

        self.class_dict = {0: 'ambulance', 1: 'fire fighting vehicle', 2: 'intensive care unit', 3: 'non emergency car', 4: 'police car'}
        self.class_dict_inverted = {'ambulance': 0, 'fire fighting vehicle': 1, 'intensive care unit': 2, 'non emergency car': 3, 'police car': 4}

        np.random.seed(seed)
        # Перестановка файлов для рандома, сид зафиксирован
        permutation = np.random.permutation(len(self.filenames))

    def __getitem__(self, idx):
        fname = self.filenames[idx]
        image = np.asarray(Image.open(fname))
        bboxes = get_txt_data(fname, self.class_dict, self.model_type, 640, 640)

        return self.transform(image=image, bboxes=bboxes)

    def __get_raw_item__(self, idx):
        fname = self.filenames[idx]
        # TODO
        return fname, get_txt_data(fname, self.class_dict, self.model_type, 640, 640)

    def __len__(self):
        return len(self.filenames)

In [39]:
# Определение нормализации и аугментаций изображений для обучения
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_transform_yolo = A.Compose(
    [
        A.Normalize(mean=mean, std=std),
        A.ColorJitter(brightness=.5, hue=.3), 
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        A.GaussianBlur(blur_limit=(3, 7), p=0.5),
        A.Posterize(num_bits=4, p=0.5),
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5),
        ToTensorV2(),
    ],
    bbox_params=dict(format="yolo", min_visibility=0.3),
)

test_transform_yolo = A.Compose(
    [
        A.augmentations.transforms.Normalize(mean=mean, std=std),
        ToTensorV2(),
        
    ],
    bbox_params=dict(format="yolo", min_visibility=0.5),
)

# train_transform_voc = A.Compose(
#     [
#         A.augmentations.transforms.Normalize(mean=mean, std=std),
#         ToTensorV2(),
#     ],
#     bbox_params=dict(format="pascal_voc", min_visibility=0.3),
# )

# test_transform_voc = A.Compose(
#     [
#         A.augmentations.transforms.Normalize(mean=mean, std=std),
#         ToTensorV2(),
        
#     ],
#     bbox_params=dict(format="pascal_voc", min_visibility=0.5),
# )

# Применение аугментаций и создание датасетов для обучения и тестирования
# train_ds = MyDataset(root=data_path, transform=train_transform_voc, data_type_path="train", model_type=model_type)
# test_ds = MyDataset(root=data_path, transform=test_transform_voc, data_type_path="test", model_type=model_type)
# val_ds = MyDataset(root=data_path, transform=test_transform_voc, data_type_path="valid", model_type=model_type)

train_ds = MyDataset(root=data_path, transform=train_transform_yolo, data_type_path="train", model_type=model_type)
test_ds = MyDataset(root=data_path, transform=test_transform_yolo, data_type_path="test", model_type=model_type)
val_ds = MyDataset(root=data_path, transform=test_transform_yolo, data_type_path="valid", model_type=model_type)


# print(train_ds.__getitem__(0)["image"].shape)

In [15]:
print(train_ds.__getitem__(0)["image"].shape)

(82, 112, 609, 608, 1)
(37, 328, 86, 445, 3)
torch.Size([3, 640, 640])


In [40]:
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=7)
train_dataloader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=3, persistent_workers=False, pin_memory=True)
val_dataloader = DataLoader(val_ds, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=3, persistent_workers=False, pin_memory=True)
test_dataloader = DataLoader(test_ds, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=3, persistent_workers=False, pin_memory=True)

In [11]:
import wandb
wandb.login()

wandb: Currently logged in as: miraellax. Use `wandb login --relogin` to force relogin


True

In [41]:
class MyModule(LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model([x])

    def configure_optimizers(self):
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate)
        self.scheduler = torch.optim.lr_scheduler.LinearLR(self.optimizer, start_factor=0.01)
        # self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=30, gamma=0.8)
        return {"optimizer": self.optimizer, "lr_scheduler": self.scheduler}

    def training_step(self, train_batch, batch_idx):
        self.model.train()
        print("train batch", train_batch)
        images, bboxes = train_batch[0], train_batch[1]
        pred = self.model(images)

        with torch.no_grad():
            self.model.eval()
            loss_dict = model(images, pred)
            # print("loss", loss_dict)
            # wandb лог
            self.log('train_loss ', loss_dict, on_step=False, on_epoch=True)
            print("Train loss: ", loss_dict)

        # step every x epochs
        if self.trainer.is_last_batch and (self.trainer.current_epoch + 1) % 2 == 0:
          self.scheduler.step()

        # # В конце эпохи сохраняем модель
        # if self.trainer.is_last_batch:
        #     accuracy = evaluate_task(self.model, val_dataloader)
        #     # weights = self.model.state_dict()
        #     name = f"49_acc_{np.clip(10 * (accuracy.cpu() - 0.5) / 0.34, 0, 10):.2f}_epoch{self.trainer.current_epoch + 1}"
        #     name = name.replace(".", "_")
        #     trainer.save_checkpoint(name)
        #     wandb.save(name)
        #     print(name)
        
        return loss

    def validation_step(self, val_batch, batch_idx):
        # print("val batch", val_batch)
        with torch.no_grad():
            self.model.eval()
            images, target = val_batch["image"], val_batch["bboxes"]
            pred = self.model(images)
            loss_dict = model(images, pred)
            # wandb лог
            self.log('val_loss ', loss, on_step=False, on_epoch=True)
            print("Val loss: ", loss)

In [83]:
# # from torchvision.models import resnet18, resnet50, resnet101
# from torchvision.models.detection import ssd300_vgg16
# seed_everything(seed)

# # {0: 'ambulance', 1: 'fire fighting vehicle', 2: 'intensive care unit', 3: 'non emergency car', 4: 'police car'}
# MyNet = ssd300_vgg16(num_classes=5)
# # print(MyNet)

In [42]:
wandb.init(project="Emergency_Cars" )

In [43]:
import ultralytics
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

wandb.watch(model)

[]

In [14]:
# # SSD
# from roboflow import Roboflow
# rf = Roboflow(api_key="pZoUBHM37JhKZ8lshq3X")
# project = rf.workspace("emergency-vehicles-russia").project("emergency-vehicles-russia")
# version = project.version(3)
# dataset = version.download("voc")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Emergency-Vehicles-Russia-3 in voc:: 100%|████| 873/873 [00:00<00:00, 1264.16it/s]


In [1]:
# YOLO v8
from roboflow import Roboflow
rf = Roboflow(api_key="pZoUBHM37JhKZ8lshq3X")
project = rf.workspace("emergency-vehicles-russia").project("emergency-vehicles-russia")
dataset = project.version(3).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.1, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Emergency-Vehicles-Russia-3 in yolov8:: 100%|██████████████████████████| 880/880 [00:00<00:00, 1478.24it/s]


In [8]:
!yolo task=detect mode=train model=yolov8m.pt data="C:\\Users\\Alex\\PycharmProjects\\Cursach3\\Emergency Vehicles Russia.v3i.yolov8\\data.yaml" epochs=100 imgsz=640 plots=True

вљ пёЏ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8m.pt...
Ultralytics YOLOv8.1.1 рџљЂ Python-3.9.13 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=C:\\Users\\Alex\\PycharmProjects\\Cursach3\\Emergency Vehicles Russia.v3i.yolov8\\data.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train22, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, aug


  0%|          | 0.00/49.7M [00:00<?, ?B/s]
  0%|          | 48.0k/49.7M [00:00<02:01, 428kB/s]
  0%|          | 96.0k/49.7M [00:00<03:59, 217kB/s]
  0%|          | 128k/49.7M [00:00<06:28, 134kB/s] 
  0%|          | 152k/49.7M [00:01<06:51, 126kB/s]
  0%|          | 168k/49.7M [00:01<07:58, 108kB/s]
  0%|          | 184k/49.7M [00:02<15:58, 54.2kB/s]
  0%|          | 200k/49.7M [00:02<16:39, 51.9kB/s]
  0%|          | 208k/49.7M [00:02<21:08, 40.9kB/s]
  0%|          | 224k/49.7M [00:03<21:42, 39.8kB/s]
  0%|          | 240k/49.7M [00:03<24:01, 36.0kB/s]
  1%|          | 256k/49.7M [00:04<30:40, 28.2kB/s]
  1%|          | 272k/49.7M [00:05<29:40, 29.1kB/s]
  1%|          | 288k/49.7M [00:06<40:18, 21.4kB/s]
  1%|          | 304k/49.7M [00:07<37:52, 22.8kB/s]
  1%|          | 320k/49.7M [00:09<1:00:38, 14.2kB/s]
  1%|          | 336k/49.7M [00:10<1:07:56, 12.7kB/s]
  1%|          | 352k/49.7M [00:11<1:01:06, 14.1kB/s]
  1%|          | 368k/49.7M [00:12<57:43, 14.9kB/s]  
  1%|        

In [16]:
# from torchvision.models.detection.ssd import SSDClassificationHead
# from torchvision.models.detection import _utils
# from torchvision.models.detection import SSD300_VGG16_Weights
# def create_model(num_classes=5, size=640):
#     # Load the Torchvision pretrained model.
#     model = torchvision.models.detection.ssd300_vgg16(
#         weights=SSD300_VGG16_Weights.COCO_V1
#     )
#     # Retrieve the list of input channels. 
#     in_channels = _utils.retrieve_out_channels(model.backbone, (size, size))
#     # List containing number of anchors based on aspect ratios.
#     num_anchors = model.anchor_generator.num_anchors_per_location()
#     # The classification head.
#     model.head.classification_head = SSDClassificationHead(
#         in_channels=in_channels,
#         num_anchors=num_anchors,
#         num_classes=num_classes,
#     )
#     # Image size for transforms.
#     model.transform.min_size = (size,)
#     model.transform.max_size = size
#     return model

# model = create_model(5, 640)

In [ ]:
# model.train(data='/content/data.yaml', epochs=3, imgsz=640, device=device)

model.train()
for batch_idx, (data, target) in enumerate(train_dataloader):
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % args.log_interval == 0:
        wandb.log({"loss": loss})

In [ ]:
# # Для обучения только последнего слоя, выключаем обучение предыдущих
# for param in YourNet.parameters():
#     param.requires_grad = False
    
# YourNet.fc = nn.Linear(2048, 200)
# YourNet.fc.requires_grad = True
# # print(YourNet)

In [62]:
wandb.init( project="Emergency_Cars" )

In [44]:
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(log_model='all')

lr = 0.1
model = MyNet.to(device)
module = MyModule(model, lr)

NameError: name 'MyNet' is not defined

In [158]:
wandb_logger.watch(model)

C:\Users\Alex\PycharmProjects\Cursach3\venv\lib\site-packages\pytorch_lightning\loggers\wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [159]:
trainer = Trainer(logger=wandb_logger, max_epochs=3, callbacks=[EarlyStopping(monitor="val_loss ", mode="min", check_on_train_epoch_end=False)])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [160]:
trainer.fit(module, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | SSD  | 24.1 M
-------------------------------
24.1 M    Trainable params
38.7 K    Non-trainable params
24.1 M    Total params
96.588    Total estimated model params size (MB)


Sanity Checking: |                                                                     | 0/? [00:00<?, ?it/s]

C:\Users\Alex\PycharmProjects\Cursach3\venv\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


(345, 210, 628, 577, 4)
(96, 278, 176, 337, 3)
(285, 274, 314, 333, 3)
(47, 2, 640, 640, 0)
(485, 9, 548, 66, 3)
(290, 229, 444, 629, 1)
(446, 37, 485, 113, 3)
(16, 96, 322, 556, 0)
(252, 205, 584, 453, 1)
(51, 212, 207, 432, 1)
(491, 322, 617, 466, 3)
(135, 33, 640, 640, 4)
(306, 306, 385, 395, 3)
(54, 214, 333, 493, 1)
(130, 122, 494, 640, 3)
(1, 272, 40, 421, 3)
(93, 291, 133, 366, 3)
(584, 319, 640, 451, 3)
(1, 55, 347, 527, 1)
(146, 51, 620, 556, 4)
(196, 145, 485, 442, 1)
(509, 276, 551, 318, 3)
(554, 269, 594, 304, 3)
(473, 277, 507, 319, 3)
(590, 270, 623, 301, 3)
(1, 209, 417, 640, 3)
(1, 5, 437, 408, 3)
(412, 32, 629, 368, 1)
(131, 177, 385, 518, 4)
(73, 334, 111, 386, 3)
(188, 94, 640, 640, 4)
(34, 451, 60, 487, 3)
(234, 154, 435, 358, 3)
(165, 137, 288, 254, 3)
(266, 199, 640, 640, 3)
(51, 1, 526, 575, 0)


NameError: name 'tensor' is not defined

In [49]:
# trainer.save_checkpoint('helpme.pth')
# wandb.save('helpme')
wandb.finish()

In [ ]:
# def annotation2txt(bboxes, w_im, h_im):
#     # (xmin, ymin, xmax, ymax, class in dict) -> [class_id center_x center_y width height]
#     res = []
#     for box in bboxes:
#       s = f"{box[4]} {((box[2] + box[0])/2)/w_im} {((box[3] + box[1])/2)/h_im} {(box[2] - box[0])/w_im} {(box[2] - box[0])/h_im}"
#       res.append(s)
#
#     return res
#
# import ultralytics
# from ultralytics import YOLO
#
# # Обучение с нуля - save_dir: PosixPath('runs/detect/train5')
# model = YOLO('yolov8n.yaml')
# model.train(data='/content/data.yaml', epochs=100, imgsz=512, device=device)
#
# # Загрузка весов из обучения выше
# # model = YOLO('runs/detect/train5/weights/best.pt')
#
# from IPython.display import Image
# Image('/content/runs/detect/train/results.png')
#
# imgs = []
# bxs = []
#
# results = model.predict(images, save=True, imgsz=[512, 512], conf=0.5, rect=True)
# for result in results:
#     box_res = []
#     # Собираем данные о бибоксах и картинки в списки для visualize
#     # bbox = (xmin, ymin, xmax, ymax, card_class)
#     boxes = result.boxes.xyxy.tolist()
#     im = torch.from_numpy(result.orig_img.astype(np.uint8))
#     im = im.permute(2, 0, 1)
#     cls = result.boxes.cls.tolist()
#
#     for box in range(len(boxes)):
#       b = (boxes[box][0], boxes[box][1], boxes[box][2],  boxes[box][3], int(cls[box]))
#       box_res.append(b)
#
#     imgs.append(im)
#     bxs.append(box_res)
#
# visualize(imgs, bxs)